In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subprocess as sp
from scipy.interpolate import interpn

In [ ]:
def read_dist(filename):
    with open(filename) as f:
        #w, h = [int(x) for x in next(f).split()] # read first line
        ys = []
        for line in f: # read rest of lines
            ys.append([int(x) for x in line.split()])
        return ys

In [ ]:
def get_compress_size(dataset, compressor="sz", nprocs=1, shape="100x500x500"):
    '''
    calls the libpressio container via runcmd wrapper
    returns the compressed size across the given commpressor and 
    process count extracted from json
    '''
    result = sp.run(f"./runcmd data_dist.py -n {nprocs} -j {dataset} --shape {shape} | jq '.{compressor}[].metrics[\"size:compressed_size\"]'",
          shell=True, check=True, capture_output=True, text=False)
    
    return [int(x) for x in str(result.stdout, encoding='utf-8').splitlines()]

In [ ]:
def plot_compress(ys):
    xs = [x for x in range(len(ys))]
    plt.plot(xs, ys)
    plt.show()
    # Make sure to close the plt object once done
    plt.close()

In [ ]:
def plot_hist(ys, bins=10):
    npa = np.asarray(ys, dtype=np.float32)
    plt.hist(npa, bins)
    plt.show()

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="100")

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-100-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('zfp-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys=read_dist('sz-25-slice')

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

## Test 1D interpolations

https://docs.scipy.org/doc/scipy/tutorial/interpolate/1D.html

In [ ]:
def gen_1d(y, orig, new):
    npy=np.array(y)
    xnew = np.linspace(0, len(npy), num=new)
    return np.interp(xnew, range(len(npy)), npy)

In [ ]:
sample=4

In [ ]:
scale=10

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs=sample)

In [ ]:
plot_compress(ys)

In [ ]:
plot_compress(gen_1d(ys, sample, scale))

In [ ]:
plot_compress(gen_1d(ys, sample, 100))

In [ ]:
sample=100

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs=sample)

In [ ]:
plot_compress(ys)

In [ ]:
sample

In [ ]:
plot_compress(gen_1d(ys, sample, 1000))

In [ ]:
plot_hist(ys, 50)

In [ ]:
plot_hist(gen_1d(ys, sample, 1000), 50)

## Tri-linear with scipy interpn

Documentation of [interpn()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interpn.html#scipy.interpolate.interpn)

Code based on example from documentation

In [ ]:
ys=get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="25")

In [ ]:
g_truth = get_compress_size(dataset="datasets/CLOUDf48.bin.f32", compressor="zfp", nprocs="100")

In [ ]:
ys=np.array(ys)

In [ ]:
yidx = np.array(range(len(ys)))

In [ ]:
yidx

In [ ]:
newy = np.linspace(0, len(ys)-1, 100)

In [ ]:
genval=interpn((yidx,), ys, newy)

In [ ]:
plot_compress(ys)

In [ ]:
plot_compress(genval)

In [ ]:
plot_compress(g_truth)

In [ ]:
plot_hist(ys)

In [ ]:
plot_hist(genval)

In [ ]:
plot_hist(g_truth)

## Cubify data set 

In [ ]:
def cubify(arr, newshape):
    '''
    Reshape 3D shape into new 3D shape
    
    Credit: https://stackoverflow.com/a/42298440
    Tutorial for reshaping: https://realpython.com/numpy-reshape/
    '''
    oldshape = np.array(arr.shape)
    repeats = (oldshape / newshape).astype(int)
    tmpshape = np.column_stack([repeats, newshape]).ravel()
    order = np.arange(len(tmpshape))
    order = np.concatenate([order[::2], order[1::2]])
    # newshape must divide oldshape evenly or else ValueError will be raised
    return arr.reshape(tmpshape).transpose(order).reshape(-1, *newshape)


In [ ]:
dataset="datasets/CLOUDf48.bin.f32"
datatype=np.float32
dataset_shape=[100,500,500]
procs=4
procs_shape=[4,5,10]

In [ ]:
pwd=globals()['_dh'][0]

In [ ]:
input_path = pwd + "/" + dataset
input_data = np.fromfile(input_path, dtype=datatype).reshape(dataset_shape)

In [ ]:
newshape=cubify(input_data, (25, 250, 250) )

In [ ]:
n, z, y, x = np.shape(newshape)

In [ ]:
newshape[0].shape

In [ ]:
ys = []

In [ ]:
for i in range(n):
    newshape[i].tofile("datasets/part.f32")
    csize=get_compress_size(dataset="datasets/part.f32", compressor="zfp", nprocs="1", shape="25x250x250")
    ys.append(csize)

In [ ]:
ys = np.array(ys)

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys.shape

In [ ]:
ys

In [ ]:
ys.reshape(2,2,4)

In [ ]:
ys.reshape(4,1,4)

In [ ]:
ysbase = ys.reshape(2,2,4)

In [ ]:
ysbase

## Explore 3D interpolation

Example from [RegularGridInterpolator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.RegularGridInterpolator.html#scipy.interpolate.RegularGridInterpolator)

In [ ]:
from scipy.interpolate import RegularGridInterpolator

import numpy as np

def f(x, y, z):

    return 2 * x**3 + 3 * y**2 - z

x = np.linspace(1, 4, 11)

y = np.linspace(4, 7, 22)

z = np.linspace(7, 9, 33)

xg, yg ,zg = np.meshgrid(x, y, z, indexing='ij', sparse=True)

data = f(xg, yg, zg)

In [ ]:
data.shape

In [ ]:
x.shape

In [ ]:
zg.shape

In [ ]:
interp = RegularGridInterpolator((x, y, z), data)

In [ ]:
interp

In [ ]:
pts = np.array([[2.1, 6.2, 8.3],

                [3.3, 5.2, 7.1]])

interp(pts)

In [ ]:
pts.shape

In [ ]:
x

In [ ]:
y

In [ ]:
z

In [ ]:
f(2.1, 6.2, 8.3), f(3.3, 5.2, 7.1)

In [ ]:
newx = np.linspace(1, 4, 22)
newy = np.linspace(4, 7, 33)
newz = np.linspace(7, 9, 44)

In [ ]:
newx

In [ ]:
nxg, nyg ,nzg = np.meshgrid(newx, newy, newz, indexing='ij', sparse=True)

In [ ]:
nxg.shape

In [ ]:
nxg

In [ ]:
npts = np.array([nxg, nyg, nzg])

In [ ]:
npts = np.linspace([1,4,7], [4,7,9], 10)

In [ ]:
npts.shape

In [ ]:
npts

In [ ]:
interp(npts)

In [ ]:
np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7]))#.shape#.T#.reshape(-1,3)

In [ ]:
np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7])).T

In [ ]:
np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7])).T.reshape(-1,3)

In [ ]:
newpts = np.array(np.meshgrid(newx, newy, newz)).T.reshape(-1,3)

In [ ]:
newpts.shape

In [ ]:
22*33*44

In [ ]:
newpts

In [ ]:
interp(newpts).reshape(22,33,44).shape

## Reshape 2x2x4 process space of compressed data

In [ ]:
plot_compress(ys)

In [ ]:
plot_hist(ys)

In [ ]:
ys

In [ ]:
# process compression (pc) results 4 slices of 2x2 processes (25,250,250) data sizes
pc_data = ys.reshape(4,2,2)

In [ ]:
pc_data

Create 4x2x2 index mesh

In [ ]:
x=np.arange(2)

y=np.arange(2)

z=np.arange(4)

create mesh grid for process compression interpolation

In [ ]:
pc_data.shape

In [ ]:
(z,x,y)

In [ ]:
c_space = RegularGridInterpolator((z, x, y), pc_data)

### Test various points in source xyz space and confirm that the input values are returned

In [ ]:
c_space([0,0,0])

In [ ]:
c_space([0,0,1])

works with floats too

In [ ]:
c_space([0.0,0.0,1.0])

In [ ]:
c_space([1,1,1])

In [ ]:
c_space([1.0,1.0,1.0])

In [ ]:
c_space([3,0,0])

In [ ]:
c_space([3.0,0.0,0.0])

on interpolation point along x-axis

In [ ]:
c_space([0.0,0.0,0.333333])

### create index points for interpolation of scale up mesh size

use linspace since we have to stay in the range of the original dimensions to interpolate values.

we can convert to integer indices after the interpolated values have been created.

In [ ]:
scale=2

In [ ]:
len(x)

In [ ]:
newx=np.linspace(x[0],x[-1],len(x)*scale)
newy=np.linspace(y[0],y[-1],len(y)*scale)
newz=np.linspace(z[0],z[-1],len(z)*scale)

In [ ]:
newx

In [ ]:
newy

In [ ]:
newz

### generate all the points to interpolate for the new mesh

In [ ]:
zslices_grid = np.array(np.meshgrid(newx, newy, newz)).T.reshape(-1,3)

In [ ]:
np.meshgrid(newz, newy, newx)

In [ ]:
np.array(np.meshgrid(newz, newy, newx))

In [ ]:
np.array(np.meshgrid(newz, newy, newx)).T#.shape

In [ ]:
np.array(np.meshgrid(newz, newx, newy)).T.reshape(-1,3)

In [ ]:
newgrid = np.array(np.meshgrid(newz, newx, newy)).T.reshape(-1,3)

In [ ]:
newgrid

#### Manually create index collection since meshgrid seems intractable

Can't get meshgrid to return the index order of z,y,x needed by the RegularGridInterpolator.

Use a for loop stack to work around this for now.

In [ ]:
darray = np.zeros((len(newx)*len(newy)*len(newz),3))

In [ ]:
i = 0

for zval in newz:
    for yval in newy:
        for xval in newx:
            darray[i] = [z, y, x]
            i+=1

In [ ]:
darray

In [ ]:
newgrid = darray

#### Confirm correct number of points are generated

We generate the correct number of points for a 2x scaled up 4x2x2 

In [ ]:
newgrid.shape

In [ ]:
len(z)*scale*len(x)*scale*len(y)*scale

In [ ]:
len(c_space(newgrid))

In [ ]:
c_space(newgrid)

### Create interpolated commpression points

In [ ]:
pc_data

In [ ]:
c_space(newgrid)

In [ ]:
gen_compress = c_space(newgrid).reshape(len(z)*scale, len(y)*scale, len(x)*scale)#.T

reference source model

In [ ]:
pc_data

In [ ]:
gen_compress

In [ ]:
gen_compress[0,3,3]

In [ ]:
plot_hist(ys)

In [ ]:
plot_hist(gen_compress.reshape(-1))

In [ ]:
plot_compress(ys)

In [ ]:
plot_compress(gen_compress.reshape(-1))

In [ ]:
len(newx)

In [ ]:
 gen_compress.ravel()[0:15]

In [ ]:
gen_compress.ravel()[0:16].reshape(4,4)

In [ ]:
newx.ravel()

In [ ]:
xy = np.array(np.meshgrid(newx,newy)).reshape(-1,2)


In [ ]:
xy

In [ ]:
xy[:,0]

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()

ax = fig.add_subplot(projection='3d')

ax.plot_surface(newx, newy, gen_compress.ravel()[0:16].reshape(4,4),)

          # s=60, c='k', label='data')


In [ ]:
X, Y = np.meshgrid(newx, newy, indexing='ij')

In [ ]:
ax.plot_wireframe(X, Y, gen_compress.reshape(-1), rstride=3, cstride=3,

                  alpha=0.4, color='m', label='linear interp')
